In [1]:
import pandas as pd
import numpy as np
import os
import csv
import ast
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
sys.path.append("../Codes/")
# import mobility as mbl
from python_mobility import parameters as mp

my package for mobility extraction has been imported!


In [2]:
###1 mobility
mobility_map_method = '1tomore'
#  loss_visitors
#  1tomore
#  1to1

In [3]:
df_flow = pd.read_csv('../data/raw_mobility/outflow_7c_wk_raw_{}.csv'.format(mobility_map_method))
df_flow['MODZCTA'] = df_flow['MODZCTA'].astype(str)

In [4]:
cluster_cols = [mp.cluster_name0[0]+'_{}'.format(i) for i in range(56)] 
# 56 weeks from 2020-02-03 (Monday) to 2021-02-28 (Sunday included) 
# only need 53 weeks from 2020-02-24 to 2021-02-028 (Sunday included)
# only need 50 weeks from 2020-03-16 (Monday) to 2021-02-028 (Sunday included)
df_vist_cluster_i = df_flow[['MODZCTA','mod_idx']+cluster_cols].copy()

final_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA','mod_idx'], value_vars = cluster_cols[6:56], var_name='week', value_name=mp.cluster_name0[0]+'_visits_weekly')
#### 
# Correctly adjust the code to perform the intended replacement
final_df['week'] = final_df['week'].apply(lambda x: (int(x.split('_')[1]) - 6))

for c_idx in range(1,7):
    print(mp.cluster_name0[c_idx])
    cluster_cols = [mp.cluster_name0[c_idx]+'_{}'.format(i) for i in range(56)]
    df_vist_cluster_i = df_flow[['MODZCTA', 'mod_idx']+cluster_cols].copy()
    temp_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA', 'mod_idx'], value_vars = cluster_cols[6:56], 
                    var_name='week', value_name=mp.cluster_name0[c_idx]+'_visits_weekly')
    temp_df['week'] = temp_df['week'].apply(lambda x: (int(x.split('_')[1]) - 6))
    final_df = pd.merge(final_df, temp_df, on=['MODZCTA','mod_idx','week'])

Retails
Arts&Entertainment
Restaurants&Bars
Educations
Healthcares
others


Merge all the other data at the zipcode level

In [5]:
### 2 read scores data at zipcode level#########
final_df['MODZCTA'] = final_df['MODZCTA'].astype(int)
df_mod_score = pd.read_csv('../results/scores/modzcta_scores.csv')
df_2 = final_df.merge(df_mod_score, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_2.dropna(subset=['score_median'],inplace=True)
print(len(df_2)) ### 6350

6350


In [ ]:
##### 3 social economic data ######
df_sed_nyc = pd.read_csv('../Data/sed_nyc_modzcta_2021.csv')
# df_sed_nyc = pd.read_csv('../Data/sed_old.csv')
# df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='right')
### when merged on the right will have the subset of zipcodes 
##  where the zipcodes counts are satisfied
print(len(df_3))
#### merge with the score mode #####
df_score_mode = pd.read_csv('../results/scores/modzcta_scores_mode.csv')
df_3 = df_3.merge(df_score_mode, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
print(len(df_3)) ### 850

850
850


In [7]:
### 4 merge with borough cases ###
df_borough = pd.read_csv('../Data/mappings/data-by-modzcta.csv')
df_borough['borough'] = df_borough['BOROUGH_GROUP'].replace({'Manhattan':'MN', 'Brooklyn':'BK', 'Queens':'QN', 'Bronx':'BX', 'Staten Island':'SI'})
df_borough_s = df_borough.rename(columns={'MODIFIED_ZCTA':'MODZCTA',
                                          'COVID_CONFIRMED_CASE_COUNT':'COVID_CONFIRMED_CASE_COUNT_final', 
                                          'COVID_PROBABLE_CASE_COUNT':'COVID_PROBABLE_CASE_COUNT_final',
                                          'COVID_CASE_COUNT':'COVID_CASE_COUNT_final',
                                          'COVID_CONFIRMED_CASE_RATE':'COVID_CONFIRMED_CASE_RATE_final', 
                                          'COVID_CASE_RATE':'COVID_CASE_RATE_final',
                                          'COVID_DEATH_COUNT':'COVID_DEATH_COUNT_final', 
                                          'COVID_DEATH_RATE':'COVID_DEATH_RATE_final' })
df_3 = df_3.merge(df_borough_s, on='MODZCTA',how='left')

In [8]:
covid_cases = pd.read_csv('../data/cases_borough_weekly.csv')
for col in ['CASE_COUNT','DEATH_COUNT', 'borough_case_count', 'borough_death_count']:
    covid_cases[col] = covid_cases[col].astype(float)
    covid_cases.loc[covid_cases[col]==0,col] = 0.1
    covid_cases[col+'_log'] = np.log(covid_cases[col])

df_4 = df_3.merge(covid_cases, on=['borough', 'week'], how='left')
print(len(df_4))

850


In [9]:
### 5 merge with policies ####
df_policies = pd.read_csv('../data/stringency_index_ny_wk.csv')
visits_scores_wk = df_4.merge(df_policies, on='week', how='left')
print(len(visits_scores_wk))

850


In [10]:
### 6 merge with spatial data ####
import geopandas as gpd
gdf = gpd.read_file('../Data/geography_data/MODZCTA_2010.shp')
gdf['centroid'] = gdf.geometry.centroid
# Extract longitude and latitude of the centroids
gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y
gdf_s = gdf[['MODZCTA','longitude','latitude']].copy()
gdf_s['MODZCTA'] = gdf_s['MODZCTA'].astype(int)
visits_scores_wk = visits_scores_wk.merge(gdf_s, on='MODZCTA',how='left')
print(len(visits_scores_wk))

850


In [11]:
# Ensure the DataFrame is sorted by MODZCTA and week
visits_scores_wk.sort_values(by=['MODZCTA', 'week'], inplace=True)
y_ls = ['Grocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly']
for col in y_ls:
    visits_scores_wk[col+'_lag1'] = visits_scores_wk.groupby('MODZCTA')[col].shift(1)

In [12]:
visits_scores_wk.to_csv('../data/unpivot_merged_data_raw_{}_s.csv'.format(mobility_map_method),index=False)

In [30]:
non_positive = visits_scores_wk[(visits_scores_wk[['Grocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly']] <= 0).any(axis=1)]